# CoreEngine Class

The CoreEngine class serves as parent to data dedicated Engine classes (e.g. MachineLearningEngine), following inheritance. Similar methods accros engines are applied to the CoreEngine for reusability. 

In [ ]:
from src.StreamPort.core.CoreEngine import CoreEngine

# Creates an empty CoreEngine object and prints it
x = CoreEngine()
x.print()

## ProjectHeaders Class

The ProjectHeaders class is used for managment of project information when using an Engine.

In [ ]:
from src.StreamPort.core.CoreEngine import CoreEngine
from src.StreamPort.core.ProjectHeaders import ProjectHeaders

x = CoreEngine()

print("1:")
# Creates a ProjectHeaders object and prints it 
ph = ProjectHeaders()
ph.print()

# Validates the ProjectHeaders object
print(ph.validate())

print("\n2:")
# Modifies headers in the CoreEngine using a dictionary
x.add_headers(headers={"name": "Example Name", "description": "demo description"})
x.print()

print(x.get_headers())

print("\n3:")
# Removes the description header
x.remove_headers("description")
print(x.get_headers())

## Analysis Class

Within the engine, data is treated as units of class Analysis. Analysis is always the parent of sub-class dedicated childs (e.g. SensorAnalysis). The Analysis holds similar methods and attributes to all Analysis class childs. 

In [1]:
from src.StreamPort.core.CoreEngine import CoreEngine
from src.StreamPort.core.Analysis import Analysis
import numpy as np

x = CoreEngine()

print("1: Creates an Analysis object and prints it")

ana1 = Analysis(name = "Analysis 1", data = {"x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)})
ana1.print()

print("Data: ")
for key, value in ana1.data.items():
  print(f"{key}: {value}")
print("\n")

print("2: Creates a list of Analysis objects and prints the number of analyses in the list")
anaList = [
  ana1,
  Analysis(name = "Analysis 2", data = {"x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)}),
  Analysis(name = "Analysis 3", data = {"x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)})
]
print("Number of analyses: ", anaList.__len__())
print("\n")

print("3: Adds the list of analyses to the CoreEngine")
x.add_analyses(anaList)
print("Number of analyses: ", x._analyses.__len__())
print("/n")

print("4: Gets the second analysis from the engine")
ana2 = x.get_analyses(1)
print(ana2)

print("5: Removes the first analysis from the engine")
x.remove_analyses(0)
print("Number of analyses: ", x._analyses.__len__())
print("/n")

print("6: Removes all analyses from the engine")
x.remove_analyses()
print("Number of analyses: ", x._analyses.__len__())
print("/n")


1: Creates an Analysis object and prints it

Analysis
  name: Analysis 1
  replicate: None
  blank: None
  data:
    x (size 10)
    y (size 10)

Data: 
x: [ 1  2  3  4  5  6  7  8  9 10]
y: [39 48 21 47 13 79 79 23 35 88]


2: Creates a list of Analysis objects and prints the number of analyses in the list
Number of analyses:  3


3: Adds the list of analyses to the CoreEngine
Number of analyses:  3


4: Gets the second analysis from the engine

Analysis
  name: Analysis 2
  replicate: None
  blank: None
  data:
    x (size 10)
    y (size 10)

5: Removes the first analysis from the engine
Number of analyses:  2


6: Removes all analyses from the engine
Number of analyses:  0




## ProcessingSettings Class

The ProcessingSettings class is used to assemble data procesing wokflows within the each engine. In the engine, the ProcessingSettings are stored as a list and the order of the ProcessingSettings dictate the order of the data processing workflow to be applied to the data in each analysis.

In [2]:
from src.StreamPort.core.CoreEngine import CoreEngine
from src.StreamPort.core.ProcessingSettings import ProcessingSettings

x = CoreEngine()

print("1: Creates a ProcessingSettings object and prints it")
settings = ProcessingSettings()
settings.print()

print("2: Adds the settings to the CoreEngine")
x.add_settings(settings)
print("Number of settings: ", x._settings.__len__())
print("/n")

print("3: Removes the settings from the CoreEngine")
x.remove_settings()
print("Number of settings: ", x._settings.__len__())
print("/n")


1: Creates a ProcessingSettings object and prints it

 ProcessingSettings
 call         None
 algorithm    None
 version      None
 software     None
 developer    None
 contact      None
 link         None
 doi          None

 parameters: empty 

2: Adds the settings to the CoreEngine
Number of settings:  1


3: Removes the settings from the CoreEngine
Number of settings:  0




#### ProcessingSettings method dispatchment

The ProcessingSettings method dispatchment is used to apply the workflow based on class hierarchy. Below, an implementation example of a class processing method and possible algorithms is given. From a processing method run, results objects are always returned. The results are then updated in the engine. When results are already present the results in the engine are used to process and not the raw data within each analysis. See the results structure in the subchapter below.

In [ ]:
from src.StreamPort.core.CoreEngine import CoreEngine
from src.StreamPort.core.ProcessingSettings import ProcessingSettings
from src.StreamPort.core.Analysis import Analysis
import numpy as np

# Processing method specific class
class NormalizeData(ProcessingSettings):
  def __init__(self):
    super().__init__()
    self.call = "normalize_data"

  def run(self):
    pass

# Algorithm specific class
class NormalizeDataMinMax(NormalizeData):
  def __init__(self):
    super().__init__()
    self.algorithm = "min_max"
  
  def run(self, engine):
    if engine._results.__len__() > 0:
      results = engine._results
    else:
      results = {}
      for analysis in engine._analyses:
        if isinstance(analysis.data, dict) and 'y' in analysis.data:
          results.update({analysis.name: analysis.data})
        else:
          print(f"Skipping {analysis.name} because its data is not a dictionary with a 'y' key.")

    for key in results:
      y = np.array(results[key]['y'])
      norm_data = (y - y.min()) / (y.max() - y.min())
      results[key].update({'y': norm_data})
    
    return results

# Algorithm specific class
class NormalizeDataSNV(NormalizeData):
  def __init__(self, liftToZero = True):
    super().__init__()
    self.algorithm = "standard_variance_normalization"
    self.parameters = {"liftToZero": liftToZero}
  
  def run(self, engine):
    if engine._results.__len__() > 0:
      results = engine._results
    else:
      results = {}
      for analysis in engine._analyses:
        if isinstance(analysis.data, dict) and 'y' in analysis.data:
          results.update({analysis.name: analysis.data})
        else:
          print(f"Skipping {analysis.name} because its data is not a dictionary with a 'y' key.")

    for key in results:
      y = np.array(results[key]['y'])
      norm_data = (y - y.mean()) / y.std()

      if self.parameters["liftToZero"]:
        norm_data += abs(norm_data.min())

      results[key].update({'y': norm_data})
    
    return results

# Create an example analysis list
ex_analyses = [
  Analysis(name = "Analysis 1", data = {"x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)}),
  Analysis(name = "Analysis 2", data = {"x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)})
]

x = CoreEngine(analyses = ex_analyses)

print("1: Prints the analyses in the CoreEngine")
for analysis in x._analyses:
  print(f"Analysis: {analysis.name}")
  for key, value in analysis.data.items():
    print(f"{key}: {value}")
  print("/n")

# Create an instance of the NormalizeDataMinMax class
norm_settings = NormalizeDataMinMax()

# Create an instance of the NormalizeDataSNV class
norm_settings2 = NormalizeDataSNV()

# Run the settings MinMax using the engine as argument
results = norm_settings.run(x)

print("2: Results from settings MinMax")
for key in results:
  print(f"Analysis: {key}")
  print(f"y: {results[key]['y']}")
  print("\n")

# Run the settings SNV using the engine as argument
results2 = norm_settings2.run(x)

print("3: Results from settings snv")
for key in results2:
  print(f"Analysis: {key}")
  print(f"y: {results2[key]['y']}")
  print("\n")

# Adds settings to the CoreEngine
x.add_settings(norm_settings)

x.print()

x.run_workflow()

print("4: Results from the engine")
for key in x._results:
  print(f"Analysis: {key}")
  print(f"y: {x._results[key]['y']}")
  print("\n")


## Results dict

After processed the data is stored in Results class objects as a differentiation from Analysis class that refers to raw data. There is not defined class for results as they mirror the data dict attribute from each analysis, holding the modified/processed data.

In [ ]:
from src.StreamPort.core.CoreEngine import CoreEngine

x = CoreEngine()

print("1: Creates a results dictionary with data from two sensors")
res1 = {"sensor1": {"x": [1, 2, 3, 4, 5], "y": [1, 2, 3, 4, 5]}, "sensor2": {"x": [1, 2, 3, 4, 5], "y": [1, 2, 3, 4, 5]}}
print(res1)
print("\n")

print("2: Adds the results to the CoreEngine")
x.add_results(res1)
print("Number of results: ", x._results.__len__())
print("\n")

print("3: Removes the results in the CoreEngine")
x.remove_results("sensors")
print("Number of results: ", x._results.__len__())
print("\n")

# Class Inheritance

Engines are created based on the class inheritance as described below.
Basic methods are reused from the core engine. 

In [ ]:
class Person:
  """Class docstrings go here."""

  def __init__(self, fname, lname):
    self.firstname = fname
    self.lastname = lname

  def printname(self):
    """Class method docstrings go here."""
    print(self.firstname, self.lastname)

  def welcome_from_parent(self):
    print("Welcome from parent class")

#Use the Person class to create an object, and then execute the printname method:

x = Person("John", "Doe")

x.printname()

class Student(Person):
  def __init__(self, fname, lname, year):
    super().__init__(fname, lname)
    self.graduationyear = year

  def welcome(self):
    print("Welcome", self.firstname, self.lastname, "to the class of", self.graduationyear)

  def printname(self):
    print(self.firstname, self.lastname, "overwritten!")

  def printname_from_parent(self):
    super().printname()

x = Student("Mike", "Olsen", 2019)

x.printname()

x.printname_from_parent()

x.welcome()

x.welcome_from_parent()

#help(Person)

# Other Code Lines

In [1]:
# from PyPDF2 import PdfMerger

# # Paths to the uploaded PDF files
# pdf1_path = 'C:/Users/apoli/My Drive/submission_docs/20240829_Projektskizzen_IUTA_UDE_ZunA.pdf'
# pdf2_path = 'C:/Users/apoli/My Drive/submission_docs/LOI_LINEG.pdf'
# pdf3_path = 'C:/Users/apoli/My Drive/submission_docs/LOI_EGLV.pdf'
# pdf4_path = 'C:/Users/apoli/My Drive/submission_docs/LOI_RV.pdf'
# pdf5_path = 'C:/Users/apoli/My Drive/submission_docs/LOI_LW.pdf'

# # Output path for the merged PDF
# output_path = 'C:/Users/apoli/My Drive/submission_docs/20240829_Projektskizzen_IUTA_UDE_ZunA_final.pdf'

# # Create a PdfMerger object
# merger = PdfMerger()

# # Append the PDFs
# merger.append(pdf1_path)
# merger.append(pdf2_path)
# merger.append(pdf3_path)
# merger.append(pdf4_path)
# merger.append(pdf5_path)

# # Write out the merged PDF
# merger.write(output_path)
# merger.close()

# output_path


'C:/Users/apoli/My Drive/submission_docs/20240829_Projektskizzen_IUTA_UDE_ZunA_final.pdf'